In [7]:
import networkx as nx
import numpy as np

def read_graph(filename):
    G = nx.Graph()
    array = np.loadtxt(filename, dtype=int)
    G.add_edges_from(array)
    return G

In [8]:
fb = read_graph("facebook_combined.txt.gz")
n = len(fb)
m = len(fb.edges())
print(n, m)

4039 88234


In [9]:
from networkx.algorithms.approximation import average_clustering
average_clustering(G, trials=1000)

NameError: name 'G' is not defined

In [11]:
def sample_path_lengths(G, nodes=None, trials=1000):
    if nodes == None:
        nodes = list(G)
    else:
        nodes = list(nodes)
    
    pairs = np.random.choice(nodes, (trials, 2))
    lengths = [nx.shortest_path_length(G, *pair) for pair in pairs]
    return lengths

In [12]:
def estimate_path_lengths(G, nodes=None, trials=1000):
    return np.mean(sample_path_lengths(G, nodes, trials))

In [13]:
C = average_clustering(fb)
L = estimate_path_lengths(fb)

In [14]:
C, L

(0.58, 3.677)

In [15]:
k = int(round(2*m/n))
k

44

In [16]:
lattice = nx.watts_strogatz_graph(n, k, 0)
print(average_clustering(lattice), estimate_path_lengths(lattice))

0.753 47.895


In [17]:
random_graph = nx.watts_strogatz_graph(n, k, 1)
print(average_clustering(random_graph), estimate_path_lengths(random_graph))

0.009 2.631


In [18]:
ws = nx.watts_strogatz_graph(n, k, 0.05, seed=15)
print(average_clustering(ws), estimate_path_lengths(ws))

0.631 3.2


In [19]:
def degrees(G):
    return [G.degree(u) for u in G]


In [20]:
from empiricaldist import Pmf


In [21]:
pmf_fb = Pmf.from_seq(degrees(fb))
pmf_fb.mean(), pmf_fb.std()

(43.69101262688785, 52.41411556737521)

In [22]:
pmf_ws = Pmf.from_seq(degrees(ws))
pmf_ws.mean(), pmf_ws.std()

(44.00000000000001, 1.4309215628189869)

In [23]:
import random

def _random_subset(repeated_nodes, k):
    targets = set()
    while len(targets) < k:
        x = random.choice(repeated_nodes)
        targets.add(x)
    return targets

In [24]:
def barabasi_albert_graph(n, k):
    G = nx.empty_graph(k)
    targets = list(range(k))
    repeated_nodes = []
    
    for source in range(k, n):
        G.add_edges_from(zip([source]*k, targets))
        
        repeated_nodes.extend(targets)
        repeated_nodes.extend([source] * k)
        
        tagets = _random_subset(repeated_nodes, k)
        
    return G

In [25]:
ba = barabasi_albert_graph(4039, 22)

In [26]:
len(ba.edges())

88374

In [35]:
def cumulative_prob(pmf, x):
    ps = [pmf[value] for value in pmf.qs if value<=x]
    return np.sum(ps)

In [36]:
cumulative_prob(pmf_fb, 25)

0.5060658578856152